In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import japanize_matplotlib
%matplotlib inline
from pprint import pprint
import warnings
warnings.simplefilter('ignore')

# 「プチ」勉強会 20230624
* 今度こそPandas。先週もPandasのMultiIndexをモチーフにしたけど主題は「スライス」と「インデックス参照」と「リスト内包表記」だったので。
* 今回も、自分で問題作って自分で解いてみる。

## 今回取り組んでみたこと

1. SSDSE-E-2023.xlsxを題材にしているが、これは都道府県のいろいろな数値をカテゴリー毎に並べたものなのでほとんど2次元。<br>
   多次元化しようにも中身が数値だから、このままではgroupbyを使う余地が乏しい。<br>
   都道府県をエリアで纏める試みもあるが、それだけではできることが限られている。各々単体でしか見ないからPandasの使いどころがない。

2. そこで、今回は、エリア分けに加えて「順位、ランキング」の概念を入れた。<br>
   同様のことがビニングでも可能だが、つまりは、数値を離散化することで多次元利用を可能にしたということ。

3. そして内容は適当だが、以下の問題を設定してみた。<br>
   Pandasでは、とにかくgroupbyをぐるぐる回してgroupbyに慣れ親しむことが大事だ。<br>
   Excelでもピボットテーブルぐるぐるが分析の基本。指が勝手に動くくらい回し続ける。どこに列名を置いたらどんな表になるか書きながらイメージできるくらい回すのだ。　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
```
     (1) 関東の平均順位が最も低い項目名と平均順位
     (2) 1位の項目がある都道府県とその項目名の一覧
     (3) 全国ベースの平均順位がエリアで一番高い都道府県とその平均順位
     (4) エリア順位が2位以下でも全国順位が5位以上の都道府県と項目名の一覧
     (5) 全項目での都道府県別全ランキング表（1位から47位まで）
```

## 「最初はグー」で、まずは普通に展開。<br>
データの取得年度は時系列じゃないので削除。あと、「全国」の行は比較に使いにくいので削除。その他は全部使う。

In [ ]:

df = pd.read_excel('SSDSE-E-2023.xlsx', header=[0, 2], index_col=[0,1])[1:]
df.index.names=['地域コード','都道府県']
df.columns.names=['項目コード','項目名']
df.head()


最初に結論的なところを。
実はいろいろ試行錯誤してここに行き着いた。そのココロは以下の通り。
```
    * groupbyを多用するので、MultiIndexのままではやりにくい。全部解いて各列のnameをカラムに縦に長ーい姿にする。
    * 同じ形から加工した方が圧倒的に扱いやすい。元の表の見てくれは関係がない。「加工するとこうなる」というイメージが大事。
```

In [ ]:
df01 = df.stack([0, 1]).reset_index().rename(columns={0:'Data'})
df01


まずは、エリアを追加。これは先週もやったsliceの応用。<br>
この形のまま列を追加していく。

In [ ]:
df01['エリア'] = ''
df01 = df01.set_index('地域コード')

area_text = """
R01000-R07000 1 北海道・東北
R08000-R14000 2 関東
R15000-R24000 3 中部・北陸・東海
R25000-R30000 4 近畿
R31000-R39000 5 中国・四国
R40000-R47000 6 九州・沖縄
"""
area_list = area_text.split('\n')[1:-1]
START = slice(1, 3, None)
STOP = slice(8, 10, None)
AREA = slice(14, None, None)

for i in area_list:
    df01.loc['R'+i[START]+'000':'R'+i[STOP]+'000', 'エリア'] = i[AREA]

df01 = df01.reset_index()
df01

今回は順位（ランキング）の概念を入れる。これでgloupbyが使える。

In [ ]:
df01['全国順位'] = ''
for item in set(df01['項目名'].values):
    rank = df01[df01['項目名'] == item]['Data'].rank(ascending=False, method='min')
    df01.loc[rank.index, '全国順位'] = rank.values

df01

ちなみに、rankメソッドは、以下ように展開した状態で使えば直感的で分かりやすいことは事実。<br>
改めて解いて畳んで名前を付ければ同じ形になる。<br>
引数は、昇順降順指定の他、ランクの付け方を指定できる。
```
method='min'
```
というのは、同率3位が2人いたら、2人とも3位で4位は空位というよくあるやつ。minに寄せるという意味かな。

In [ ]:
df_ = df
df_ = df_.rank(ascending=False, method='min').head()
df_

In [ ]:
df_.stack([0,1]).reset_index().rename(columns={0:'全国順位'})

もうひとつ順位（ランキング）を入れる。

In [ ]:
df01['エリア順位'] = ''
for area in set(df01['エリア'].values):
    for item in set(df01['項目名'].values):
        area_rank = df01[(df01['エリア'] == area) & (df01['項目名'] == item)]['全国順位'].rank(method='min')
        df01.loc[area_rank.index, 'エリア順位'] = area_rank.values

df01

これで準備完了。
```
ちなみに、最初はrankメソッドを使わずに自分で関数を作った。
rankメソッド知らなかったわけじゃないが、メソッドなんて知らなくてもやる練習。
```

先に地域（エリア）列を作り。

In [ ]:
df_ = df
df_[('A0000', '地域')] = ''
area_text = """
R01000-R07000 1_北海道東北
R08000-R14000 2_関東
R15000-R24000 3_中部・北陸・東海
R25000-R30000 4_近畿
R31000-R39000 5_中国・四国
R40000-R47000 6_九州・沖縄
"""
area_list = area_text.split('\n')[1:]
START = slice(1, 3, None)
STOP = slice(8, 10, None)
AREA = slice(14, None, None)

for i in area_list:
    df_.loc['R'+i[START]+'000':'R'+i[STOP]+'000', 'A0000'] = i[AREA]

new_columns = [df_.columns[-1]] + list(df_.columns[:-1])
df_ = df_.reindex(columns=new_columns)

こんな風に関数化した。これをfor文でrankを回してconcatすれば出来上がる寸法だ。<br>
```
関数の中身。カラム毎に逆順ソートしてrankで指定した順位の都道府県をprefに格納、続いてその都道府県が所属する地域をareaに格納している。
そしてpref_listに項目コード、項目名、順位、地域、地域コード、都道府県の順（どんな順番でもいいが・・・）に格納してデータフレームで返している。
```

In [ ]:
def pref_rank(rank, data=df_):
    pref_list = []
    for col in data.columns[1:]:
        pref = data[col].sort_values(ascending=False).index[rank-1]
        area = data.at[pref ,('A0000','地域')]
        pref_list.append((col[0], col[1], rank, area, pref[0], pref[1]))
    return pd.DataFrame(pref_list, columns=['項目コード', '項目名', '順位', '地域', '地域コード', '都道府県'])

pref_rank(1)

ちなみに、細かい話だが、関数の5行目,
```
area = data.at[pref ,('A0000','地域')]
```
に、at が使われているのが分かるだろうか・・・・。最初は普通にlocでlevel=0(外側)の参照で書いていたらこんなことに・・・。

In [ ]:
def pref_rank(rank, data=df_):
    pref_list = []
    for col in data.columns[1:]:
        pref = data[col].sort_values(ascending=False).index[rank-1]
        area = data.loc[pref[0] ,'A0000']
        pref_list.append((col[0], col[1], rank, area, pref[0], pref[1]))
    return pd.DataFrame(pref_list, columns=['項目コード', '項目名', '順位', '地域', '地域コード', '都道府県'])

pref_rank(1)

実はこうやって直すことが出来るのだが、
```
area = data.loc[pref[0] ,'A0000'].values[0][0]
```
なんでこんなことになるかというと、locは要素が一つでもDataFrameを返すのだ。だから、項目名やらなんやらインデックスの要素が文字列に・・・。<br>
at なんていつ使うの？と思っていたが、「なるほどこういう場合はatなんだ」と分かった。

In [ ]:
def pref_rank(rank, data=df_):
    pref_list = []
    for col in data.columns[1:]:
        pref = data[col].sort_values(ascending=False).index[rank-1]
        area = data.loc[pref[0] ,'A0000'].values[0][0]
        pref_list.append((col[0], col[1], rank, area, pref[0], pref[1]))
    return pd.DataFrame(pref_list, columns=['項目コード', '項目名', '順位', '地域', '地域コード', '都道府県'])

pref_rank(1)

### 寄り道した・・・。いよいよ問題を解く。(というほどのものでもないが)

```
     (1) 関東の平均順位が最も低い項目名と平均順位
     (2) 1位の項目がある都道府県とその項目名の一覧
     (3) 全国ベースの平均順位がエリアで一番高い都道府県とその平均順位
     (4) エリア順位が2位以下でも全国順位が5位以上の都道府県と項目名の一覧
     (5) 全項目での都道府県別全ランキング表（1位から47位まで）
     (6) 全国1位を除いたベースでの都道府県別全ランキング表
```

In [ ]:
#1
df01[df01['エリア'] == '2 関東'].groupby(['項目名'])[['全国順位']].mean().sort_values('全国順位', ascending=False).head(1)

なんか、簡単過ぎるような気もするが、それは準備ができているから。groupbyのこの形に持ち込めば大体のものは抽出できる。

In [ ]:
#2
df01[df01['全国順位'] == 1].groupby('都道府県')[['項目名']].sum()

んー。なんというかイマイチ。文字が繋がってしまっているし、全部表示できていないし・・・ということで、項目コードも入れてみる。

In [ ]:
df_ = df01[df01['全国順位'] == 1].groupby(['都道府県', '項目コード'])[['項目名']].sum()
df_

今度は、項目毎になったけど、もうちょっと全体がちゃんと一覧できるようにしたい。問題にも「一覧」って書いてある。
```
ということで、ここであれだな・・・・
```

In [ ]:
from collections import defaultdict
dic = defaultdict(list)

for i, pref in enumerate(df_.index):
    dic[pref[0]].append(df_['項目名'][i])

pprint(dic, compact=True)

```
いいね、いいね! defaluldictとpprintのコラボって感じだ。
東京がなんでも1位というのはしかたがないとして、他の県が1位のやつがなかなか味わい深い・・・。
```

ちなみに、defaluldictは、引数に関数を取れる。ここではlistを引数にしているのでリストに要素を追加できる仕様となっている。

さて、お次は・・・。問題を再掲。
```
     (3) 全国ベースの平均順位がエリアで一番高い都道府県とその平均順位
     (4) エリア順位が2位以下でも全国順位が5位以上の都道府県と項目名の一覧
     (5) 全項目での都道府県別全ランキング表（1位から47位まで）
     (6) 全国1位を除いたベースでの都道府県別全ランキング表
```

In [ ]:
#3
df_ = pd.DataFrame()
for area in set(df01['エリア']):
    df_temp = df01[df01['エリア'] == area].groupby(['エリア','都道府県'])[['全国順位']].mean().sort_values('全国順位').head(1)
    df_ = pd.concat([df_, df_temp])
df_.sort_index()

これは、全国順位をエリア内で比較しているというところがちょっと捻っている。

In [ ]:
#4
df_ = df01[(df01['エリア順位'] > 1) & (df01['全国順位'] <= 5)].groupby(['都道府県','項目コード'])[['項目名']].sum()

from collections import defaultdict
dic = defaultdict(list)

for i, pref in enumerate(df_.index):
    dic[pref[0]].append(df_['項目名'][i])

pprint(dic, compact=True)

せっかくだから、ここもdefalutdict + pprintで。

あとひとつ。
```
     (5) 全項目での都道府県別全ランキング表（1位から47位まで）
```

まあなんだ。これをまた変なグラフにしたりして遊ぶと楽しみながら学習できるかもね。<br>
ランキングで文字の大きさを変えたり、色変えたり・・・

In [ ]:
#5
df01.groupby(['全国順位', '項目コード', '項目名'])[['都道府県']].sum().unstack([1, 2])


## 以上！